## A intenção do projeto é criar um chatbot baseado em reviews de filmes para que se possa fazer perguntas e manter uma conversa livre sobre este tema

- link do banco de dados https://www.kaggle.com/Cornell-University/movie-dialog-corpus?select=movie_lines.tsv
- referências
>- https://shanebarker.com/blog/deep-learning-chatbot/
> -https://towardsdatascience.com/how-to-create-a-chatbot-with-python-deep-learning-in-less-than-an-hour-56a063bdfc44

In [107]:
import string
import nltk
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import pandas as pd
import re
import numpy as np
import math
import random

[nltk_data] Downloading package wordnet to /home/douglas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Opening movie reviews

In [111]:
messages = pd.read_csv('./chatdata/movie_lines_normalized.tsv', header = None, delimiter="\t", quoting=3, encoding='ISO-8859-2')

In [112]:
messages.columns = ['msg_line', 'user_id', 'movie_id', 'msg']

In [113]:
messages.head(10)

,msg_line,user_id,movie_id,msg
0,L1045,u0,m0,They do not!
1,L1044,u2,m0,They do to!
2,L985,u0,m0,I hope so.
3,L984,u2,m0,She okay?
4,L925,u0,m0,Let's go.
5,L924,u2,m0,Wow
6,L872,u0,m0,Okay -- you're gonna need to learn how to lie.
7,L871,u2,m0,No
8,L870,u0,m0,"""""""I'm kidding. You know how sometimes you jus..."
9,L869,u0,m0,Like my fear of wearing pastels?


In [114]:
messages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304713 entries, 0 to 304712
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   msg_line  304713 non-null  object
 1   user_id   304713 non-null  object
 2   movie_id  304713 non-null  object
 3   msg       304713 non-null  object
dtypes: object(4)
memory usage: 9.3+ MB


In [115]:
messages.describe()

,msg_line,user_id,movie_id,msg
count,304713,304713,304713,304713
unique,304713,9035,659,265277
top,L212275,u4525,m289,What?
freq,1,537,1530,1679


### Cleaning the msg_line of the conversations

In [116]:
#remove charactes
def remove_char(txt):
    return re.sub('[^0-9]','', txt)

In [117]:
#leaving just the number of the index, so L872 changes to 872
messages['msg_line_clean'] = [remove_char(msg) for msg in messages['msg_line']]

In [118]:
#change the column type to number
messages['msg_line_clean'] = pd.to_numeric(messages['msg_line_clean'])

In [119]:
messages = messages.sort_values(by=['msg_line_clean'])

In [120]:
#set the column as the index
messages = messages.set_index('msg_line_clean')

In [121]:
messages.head(10)

,msg_line,user_id,movie_id,msg
msg_line_clean,,,,
49,L49,u0,m0,Did you change your hair?
50,L50,u3,m0,No.
51,L51,u0,m0,You might wanna think about it
59,L59,u9,m0,I missed you.
60,L60,u8,m0,It says here you exposed yourself to a group o...
61,L61,u9,m0,It was a bratwurst. I was eating lunch.
62,L62,u8,m0,With the teeth of your zipper?
63,L63,u7,m0,You the new guy?
64,L64,u2,m0,So they tell me...


### Removing entities

In [122]:
entities = pd.read_csv('./chatdata/entity_list.tsv', header = None, delimiter="\t", quoting=3, encoding='ISO-8859-2')

In [123]:
entities.columns = ['ent', 'type']

In [124]:
entities.head()

,ent,type
0,Kinda,ORG
1,The Dallas Times Herald,ORG
2,Queen Louisa,PERSON
3,A.M,GPE
4,Cousin Hop,PERSON


In [125]:
entities['ent_len'] = [len(e) for e in entities['ent']]

In [126]:
s = entities['ent_len'].sort_values(ascending=False).index

In [127]:
entities = entities.reindex(s)

In [128]:
entities = entities.reset_index(drop=True)

In [129]:
entities.head()

,ent,type,ent_len
0,"""""""How can the Bolshevik cause gain respect am...",WORK_OF_ART,237
1,"""""""The Premier wishes to inform the Government...",WORK_OF_ART,192
2,""""""" Come Tuesday twelve a.m. bingo these like-...",WORK_OF_ART,182
3,"""""""The suggestion of the President regarding t...",WORK_OF_ART,155
4,"""""""The Management of Boyd's takes pleasure in ...",WORK_OF_ART,146


In [60]:
data = messages['msg']

In [61]:
ent_list =  ['PERSON', 'ORG', 'NORP', 'FAC', 'GPE', 'LOC', 'PRODUCT', 'EVENT', 'WORK_OF_ART', 'LAW', 'LANGUAGE']
#ent_list =  ['LANGUAGE']

ent = list()
for i in range(len(entities.index)):
    if entities['type'][i] in ent_list:
        ent.append(entities['ent'][i])

In [62]:
ent = list(set(ent))
print(len(ent))
print(ent)

28514
['Seagrave Cosgrove', 'the New York Stock Exchange', 'the National Security Agency', 'Oatman', 'Yes Bob', 'Dolores Fuller', 'Hallo Peter', 'The Kiber Crystal', 'Von der Null', 'A.M.', 'Belle', 'Knowin', 'El Toro de Oro', 'the Military Police', 'Hamidou', 'P. Bugs', 'FIS', 'the Royal Kraal', 'Malcom X.', 'The Charnier Shipping Company of Marseilles France', 'Leevio Valli', 'Afganistan', 'Visa', 'Young Lochinvar', 'Veterans', 'Blow', 'Errol Flynn', 'Sgt', 'Newspaper', "you're-", 'Earth</u', 'Dread Pirate Roberts', 'the Night Train', 'Dimitri Koros', 'Jon Osterman', "Snow's Island", 'Tony Lacey', 'Go to the Stardust Ballroom', 'Good-bye Peter', 'Noguchi', '"""Old Macdonald... had --"""', 'Sonoco', 'Oracle', 'Jim itÄ\x82Â\x84Ă\x82Â\x82Ä\x82Â\x82Ă\x82Â\xads', 'Broussard', 'chairĂ\x84Â\x82Ă\x82Â\x82Ä\x82Â\x82Ă\x82Â\x97and', 'Peter Reeves', 'Green Hornet', 'Pearlstine', 'Blue Rose', "Missus Cowley's", 'Lazlo Zilagyi', 'Underground', 'Riki', 'Goddamnit Valentine', 'Jewish', 'Max Karen', 

In [63]:
dict = {}
for n in ent:
    dict[n] = len(n)

In [64]:
#sort dict by biggest values
dict = {k: v for k, v in sorted(dict.items(), key=lambda item: item[1], reverse=True)}

In [65]:
dict

{'"""How can the Bolshevik cause gain respect among the Moslems if your three representatives Buljanoff Iranoff and Kopalski get so drunk that they throw a carpet out of their hotel window and complain to the management that it didn\'t fly"': 237,
 '"""The Premier wishes to inform the Government of the United States that it will be impossible for him to attend the meeting suggested by the President unless the meeting is held in Moscow."""': 192,
 '""" Come Tuesday twelve a.m. bingo these like-minded deviates log on and start yakking it up: explicit sex crime gossip who did what to whom who wants to do what when why and how."""': 182,
 '"""The suggestion of the President regarding the possibility of a meeting in Moscow would be unacceptable to Her Majesty\'s Government at the present time."': 155,
 '"""The Management of Boyd\'s takes pleasure in requesting the company of Mr. Richard Starkey that\'s you in their recently refinished gaming rooms."': 146,
 '"""Well Jim I says, it makes me 

In [66]:
def remove_entity(corpus):
    corpus = corpus.split(' ')
    corpus = [c for c in corpus if c not in list(dict.keys())]
    return ' '.join(corpus)    

In [67]:
#chepoint
messages.to_csv('./chatdata/movie_lines_pre_processed.tsv', index=False, sep='\t', header=True)

### Opening conversation sequence

In [68]:
#read the file with the conversation sequence
conv_seq = pd.read_csv('./chatdata/movie_conversations.tsv', header = None, delimiter="\t", quoting=3, encoding='ISO-8859-2')

In [69]:
conv_seq.columns = ['user1_id', 'user2_id', 'movie_id', 'sequence']

In [70]:
conv_seq.head(10)

,user1_id,user2_id,movie_id,sequence
0,u0,u2,m0,['L194' 'L195' 'L196' 'L197']
1,u0,u2,m0,['L198' 'L199']
2,u0,u2,m0,['L200' 'L201' 'L202' 'L203']
3,u0,u2,m0,['L204' 'L205' 'L206']
4,u0,u2,m0,['L207' 'L208']
5,u0,u2,m0,['L271' 'L272' 'L273' 'L274' 'L275']
6,u0,u2,m0,['L276' 'L277']
7,u0,u2,m0,['L280' 'L281']
8,u0,u2,m0,['L363' 'L364']
9,u0,u2,m0,['L365' 'L366']


In [71]:
conv_seq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83097 entries, 0 to 83096
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user1_id  83097 non-null  object
 1   user2_id  83097 non-null  object
 2   movie_id  83097 non-null  object
 3   sequence  83097 non-null  object
dtypes: object(4)
memory usage: 2.5+ MB


In [72]:
conv_seq.describe()

,user1_id,user2_id,movie_id,sequence
count,83097,83097,83097,83097
unique,5420,5608,617,83097
top,u4331,u1475,m289,['L254941' 'L254942' 'L254943' 'L254944']
freq,193,187,338,1


### Build conversation sequence

In [73]:
def split_conversation(txt):
    txt_alt = txt.split(' ')
    return txt_alt

In [74]:
def seq_to_list(seq):
    seq_list = [remove_char(s) for s in seq]
    return seq_list

In [75]:
#initializing the msg_2 column
messages['msg_2'] = '-'

In [76]:
def link_conversations(seq_list, df, filter1, filter2):
    i = 0
    while i in range(len(seq_list)):
        if i+1 < len(seq_list):
            next_msg = df.loc[int(seq_list[i+1]), filter1]
            df.at[int(seq_list[i]), filter2] = next_msg
        i+=1

In [77]:
#link each message with its answer
for conv in conv_seq['sequence']:
    #split each sequence by space
    seq = split_conversation(conv)

    #remove the char L from the sequences
    txt_alt = [remove_char(s) for s in seq]

    #use the conversation sequence to build the target answer for each message
    link_conversations(txt_alt, messages, 'msg_pre_processed', 'msg_2')

KeyError: 663

In [78]:
messages.head(30)

,msg_line,user_id,movie_id,msg,msg_2,msg_pre_processed,target
msg_line_clean,,,,,,,
49.0,L49,u0,m0,Did you change your hair?,-,No.,1
50.0,L50,u3,m0,No.,-,You might think about it,0
51.0,L51,u0,m0,You might wanna think about it,-,maybe...,0
59.0,L59,u9,m0,I missed you.,-,It says here you exposed yourself to a group o...,0
60.0,L60,u8,m0,It says here you exposed yourself to a group o...,-,It was a bratwurst. I was eating lunch.,0
61.0,L61,u9,m0,It was a bratwurst. I was eating lunch.,-,With the teeth of your zipper?,0
62.0,L62,u8,m0,With the teeth of your zipper?,-,can you explain it better?,1
63.0,L63,u7,m0,You the new guy?,-,So they tell me...,1
64.0,L64,u2,m0,So they tell me...,-,C'mon. I'm supposed to give you the tour.,0


## Pre processing the msg

In [79]:
data = messages['msg_pre_processed']

In [80]:
lemmatizer = WordNetLemmatizer()
def pre_processing_text(corpus):
    #remove html tags
    corpus = re.sub(r'<.*?>', '', str(corpus))
    
    #remove non-alphanumeric characters
    corpus = re.sub(r'[^a-z A-Z 0-9 \s]', '', str(corpus))
    
    #remove duplicated spaces
    corpus = re.sub(r' +', ' ', str(corpus))
    
    #capitalization
    corpus = corpus.lower()
    
    #tokenization
    corpus = re.findall(r"\w+(?:'\w+)?|[^\w\s]", corpus)
    
    #lammatization
    corpus = [lemmatizer.lemmatize(c) for c in corpus]
    
    #remove punctuation
    corpus = [t for t in corpus if t not in string.punctuation]
    
    #remove stopwords
    #it makes the model worst
    stopwords_ = stopwords.words("english")
    corpus = [t for t in corpus if t not in stopwords_]
    
    corpus = ' '.join(corpus)

    return corpus

In [81]:
%%time
data_pre_processed = [pre_processing_text(str(m)) for m in data]
data_pre_processed

CPU times: user 1min 7s, sys: 47 s, total: 1min 54s
Wall time: 1min 56s


['',
 'might think',
 'maybe',
 'say exposed group freshman girl',
 'wa bratwurst wa eating lunch',
 'teeth zipper',
 'explain better',
 'tell',
 'cmon im supposed give tour',
 'talk',
 'actually howd',
 'wa kidding people actually live',
 'yeah couple outnumbered cow though',
 'many people old school',
 'thirtytwo',
 'get',
 'many people go',
 'couple thousand evil',
 'talk',
 'yeah guy never seen horse eastwood',
 'need think',
 'burn pine perish',
 '',
 'stratford sophomore even think',
 '',
 'could start haircut doesnt matter shes allowed date older sister doe thats impossibility',
 'maybe',
 'expressing opinion terrorist action',
 'well yes compared choice expression year today event quite mild way rictors gonad retrieval operation went quite well case youre interested',
 'still maintain kicked ball wa merely spectator',
 'point people perceive somewhat',
 'talk',
 'believe heinous bitch term used often',
 'need think',
 'random skid',
 'thats verona one wa gone year heard wa porn

In [82]:
messages['msg_pre_processed'] = data_pre_processed

### Checking for duplicated messages in msg

In [83]:
data = messages['msg_pre_processed']

In [84]:
dict = {}
for n in data:
    if n in dict:
        dict[n] = dict[n] + 1
    else:
        dict[n] = 1

In [85]:
#sort dict by biggest values
dict = {k: v for k, v in sorted(dict.items(), key=lambda item: item[1], reverse=True)}

In [86]:
dict

{'talk': 17547,
 'maybe': 17505,
 'explain better': 17499,
 'need think': 17284,
 '': 7895,
 'nan': 2024,
 'yes': 1571,
 'yeah': 1212,
 'know': 610,
 'right': 461,
 'okay': 439,
 'dont know': 439,
 'mean': 303,
 'thank': 283,
 'really': 277,
 'like': 258,
 'oh': 255,
 'course': 252,
 'say': 242,
 'yes sir': 232,
 'dont': 230,
 'sir': 229,
 'thanks': 225,
 'think': 223,
 'come': 216,
 'sure': 215,
 'see': 213,
 'well': 212,
 'whats': 203,
 'want': 198,
 'talking': 190,
 'tell': 189,
 'wa': 189,
 'go': 182,
 'get': 181,
 'cant': 179,
 'thats right': 170,
 'im sorry': 167,
 'uhhuh': 164,
 'going': 143,
 'nothing': 134,
 'im': 130,
 'guess': 128,
 'happened': 122,
 'one': 115,
 'thats': 115,
 'way': 111,
 'please': 110,
 'shit': 108,
 'love': 107,
 'look': 102,
 'told': 101,
 'didnt': 100,
 'dont think': 98,
 'excuse': 96,
 'good': 94,
 'time': 92,
 'dont understand': 89,
 'would': 85,
 'got': 84,
 'ok': 84,
 'said': 83,
 'sorry': 83,
 'much': 82,
 'believe': 75,
 'jesus': 73,
 'hell': 73,

In [87]:
#example of duplcated msg
messages[messages['msg_pre_processed'] == 'did you change your hair']

,msg_line,user_id,movie_id,msg,msg_2,msg_pre_processed,target
msg_line_clean,,,,,,,


In [88]:
#get the repeated messages
d_list = list()
for k in dict:
    if dict[k] > 1:
        d_list.append(k)

In [89]:
d_list

['talk',
 'maybe',
 'explain better',
 'need think',
 '',
 'nan',
 'yes',
 'yeah',
 'know',
 'right',
 'okay',
 'dont know',
 'mean',
 'thank',
 'really',
 'like',
 'oh',
 'course',
 'say',
 'yes sir',
 'dont',
 'sir',
 'thanks',
 'think',
 'come',
 'sure',
 'see',
 'well',
 'whats',
 'want',
 'talking',
 'tell',
 'wa',
 'go',
 'get',
 'cant',
 'thats right',
 'im sorry',
 'uhhuh',
 'going',
 'nothing',
 'im',
 'guess',
 'happened',
 'one',
 'thats',
 'way',
 'please',
 'shit',
 'love',
 'look',
 'told',
 'didnt',
 'dont think',
 'excuse',
 'good',
 'time',
 'dont understand',
 'would',
 'got',
 'ok',
 'said',
 'sorry',
 'much',
 'believe',
 'jesus',
 'hell',
 'wont',
 'kidding',
 'great',
 'stop',
 'god',
 'let go',
 'yet',
 'exactly',
 'dont believe',
 'never',
 'understand',
 'whats wrong',
 'wait',
 'forget',
 'im sure',
 'fuck',
 'help',
 'man',
 'huh',
 'doe',
 'night',
 'hello',
 'long',
 'hey',
 'fine',
 'money',
 'absolutely',
 'alright',
 'nope',
 'could',
 'dead',
 'else',
 

In [90]:
messages = messages.drop_duplicates(subset=['msg_pre_processed'])

In [91]:
#example of duplcated msg
messages[messages['msg_pre_processed'] == 'did you change your hair']

,msg_line,user_id,movie_id,msg,msg_2,msg_pre_processed,target
msg_line_clean,,,,,,,


In [92]:
messages.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 137436 entries, 49.0 to nan
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   msg_line           137436 non-null  object
 1   user_id            137436 non-null  object
 2   movie_id           137436 non-null  object
 3   msg                137436 non-null  object
 4   msg_2              137436 non-null  object
 5   msg_pre_processed  137436 non-null  object
 6   target             137436 non-null  object
dtypes: object(7)
memory usage: 8.4+ MB


In [93]:
messages.describe()

,msg_line,user_id,movie_id,msg,msg_2,msg_pre_processed,target
count,137436,137436,137436,137436,137436,137436,137436
unique,137436,8722,618,137436,16,137436,5
top,L638455,u4525,m238,All right we may be insignificant imperfect cr...,-,teresa,0
freq,1,317,732,1,137412,1,48368


### Removing nan msg origined by '' messages

In [131]:
#filling the nan messages with a string- not necessary
messages = messages.fillna('UNKNOWN')

### Removing apostrophes (need for embedding)

In [95]:
#The quality of the model became worst
messages['msg_pre_processed'] = [ word.replace("\'","") for word in messages['msg_pre_processed']]

<ipython-input-95-9d2f2e70cc0e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  messages['msg_pre_processed'] = [ word.replace("\'","") for word in messages['msg_pre_processed']]


### Filling '-' messages with a generic one

In [96]:
#return generic answer
def generic_answer(txt):
  asw_list = ['talk more about it',
              'can you explain it better?',
              'I need to think more about it',
              'maybe...'
              ]
  if txt == '-':
    return random.choice(asw_list)
  return txt

In [97]:
#seting a generic answer to the messages without answer
messages['msg_2'] = [generic_answer(msg) for msg in messages['msg_2']]

<ipython-input-97-416b64f870a1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  messages['msg_2'] = [generic_answer(msg) for msg in messages['msg_2']]


In [98]:
messages.head(30)

,msg_line,user_id,movie_id,msg,msg_2,msg_pre_processed,target
msg_line_clean,,,,,,,
49.0,L49,u0,m0,Did you change your hair?,maybe...,,1
50.0,L50,u3,m0,No.,maybe...,might think,0
51.0,L51,u0,m0,You might wanna think about it,maybe...,maybe,0
59.0,L59,u9,m0,I missed you.,can you explain it better?,say exposed group freshman girl,0
60.0,L60,u8,m0,It says here you exposed yourself to a group o...,can you explain it better?,wa bratwurst wa eating lunch,0
61.0,L61,u9,m0,It was a bratwurst. I was eating lunch.,I need to think more about it,teeth zipper,0
62.0,L62,u8,m0,With the teeth of your zipper?,can you explain it better?,explain better,1
63.0,L63,u7,m0,You the new guy?,I need to think more about it,tell,1
64.0,L64,u2,m0,So they tell me...,can you explain it better?,cmon im supposed give tour,0


### Tagging the msg with classes

In [99]:
def define_target(corpus):
    
    if '?' in corpus:
        return 1
    else:
        return 0

In [100]:
data = messages['msg']

In [101]:
messages['target'] = [define_target(m) for m in data]

<ipython-input-101-8c498f5d1460>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  messages['target'] = [define_target(m) for m in data]


In [102]:
messages['target'] = messages['target'].astype(int)

<ipython-input-102-359a35de7aa5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  messages['target'] = messages['target'].astype(int)


In [103]:
messages.head(20)

,msg_line,user_id,movie_id,msg,msg_2,msg_pre_processed,target
msg_line_clean,,,,,,,
49.0,L49,u0,m0,Did you change your hair?,maybe...,,1
50.0,L50,u3,m0,No.,maybe...,might think,0
51.0,L51,u0,m0,You might wanna think about it,maybe...,maybe,0
59.0,L59,u9,m0,I missed you.,can you explain it better?,say exposed group freshman girl,0
60.0,L60,u8,m0,It says here you exposed yourself to a group o...,can you explain it better?,wa bratwurst wa eating lunch,0
61.0,L61,u9,m0,It was a bratwurst. I was eating lunch.,I need to think more about it,teeth zipper,0
62.0,L62,u8,m0,With the teeth of your zipper?,can you explain it better?,explain better,1
63.0,L63,u7,m0,You the new guy?,I need to think more about it,tell,1
64.0,L64,u2,m0,So they tell me...,can you explain it better?,cmon im supposed give tour,0


### Save data

In [104]:
messages.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 137436 entries, 49.0 to nan
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   msg_line           137436 non-null  object
 1   user_id            137436 non-null  object
 2   movie_id           137436 non-null  object
 3   msg                137436 non-null  object
 4   msg_2              137436 non-null  object
 5   msg_pre_processed  137436 non-null  object
 6   target             137436 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 8.4+ MB


In [105]:
messages

,msg_line,user_id,movie_id,msg,msg_2,msg_pre_processed,target
msg_line_clean,,,,,,,
49.0,L49,u0,m0,Did you change your hair?,maybe...,,1
50.0,L50,u3,m0,No.,maybe...,might think,0
51.0,L51,u0,m0,You might wanna think about it,maybe...,maybe,0
59.0,L59,u9,m0,I missed you.,can you explain it better?,say exposed group freshman girl,0
60.0,L60,u8,m0,It says here you exposed yourself to a group o...,can you explain it better?,wa bratwurst wa eating lunch,0
...,...,...,...,...,...,...,...
666520.0,L666520,u9034,m616,Well I assure you Sir I have no desire to crea...,can you explain it better?,assure fact id obliged best advice scout seen,0
666521.0,L666521,u9030,m616,And I assure you you do not In fact I'd be obl...,talk more about it,far scout report small impi farther north,1
666546.0,L666546,u9027,m616,Splendid site Crealock splendil I want to esta...,maybe...,certainly sin,0


In [132]:
messages.to_csv('./chatdata/movie_lines_pre_processed.tsv', index=False, sep='\t', header=True)